                                                            |Pre processing data|

1. Importing Libraries and defining certain constants

In [1]:
import librosa
import os
import json
import numpy as np

DATASET_PATH = 'voiceAuth'
JSON_PATH = "auth.json"
SAMPLES_TO_CONSIDER = 22050 #equivalent to 1 second



2. Preparing the dataset

In [2]:
def prepare_dataset(dataset_path, json_path, n_mfcc=13, hop_length=512, n_fft=2048):
    print("wtf12")
    #data dictionary
    data = {
        "mappings": [], #ID?
        "labels_gender": [], #either 0 or 1 for female or male
        "labels_ID": [],
        "MFCCs": [],
        "files":[]
    }
    
    #loop through all the sub-dirs
    for f in os.listdir(dataset_path):
        f = os.path.join(dataset_path, f)
        print("wtf1")
            
        # get file path
        print("file path", f)
                
        # load audio file
        signal, sr = librosa.load(f)
                
        # ensure the audio file at least 1 sec
        if len(signal) >= 3*SAMPLES_TO_CONSIDER:
            # splitting the name
            filename_extracted = f.split('_')
            label_gender = filename_extracted[3].split(".")[0]
            
            # 0 for male and 1 for female
            if label_gender == 'male':
                label_gender = 0
            else:
                label_gender = 1
            labels = [label_gender,filename_extracted[2]]
        
            print(labels)
            
            for i in range(0,2):
                # enforce 1 sec. long signal
                signal = signal[SAMPLES_TO_CONSIDER*(i):SAMPLES_TO_CONSIDER*(i+1)]
                # extract the MFCCs
                MFCCs = librosa.feature.mfcc(y=signal, n_mfcc=n_mfcc, hop_length=hop_length,n_fft=n_fft)
                MFCCs = np.mean(MFCCs.T, axis=0)
            
                # store data
                data["labels_gender"].append(labels[0])
                data["labels_ID"].append(labels[1])
                data["MFCCs"].append(MFCCs.T.tolist())
                data["files"].append(f)
            
            
            
    # store in json file
    with open(json_path, "w") as fp:
        json.dump(data, fp, indent=4)
    

In [3]:
prepare_dataset(DATASET_PATH, JSON_PATH)

wtf12
wtf1
file path voiceAuth\HW1_Q4_810102087_female.mp3
[1, '810102087']


NameError: name 'signal_smp1' is not defined